In [2]:
!pip install wordcloud
!pip install TextBlob
!pip install plotly
!pip install statsmodels
!pip install numpy
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install nltk
!pip install scikit-learn
!pip install xgboost


In [3]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 100
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
warnings.filterwarnings("ignore")
%matplotlib inline
import re

from textblob import TextBlob
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords=set(stopwords.words('english'))
import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
import statsmodels.api as sm

from sklearn.svm import SVR
import xgboost as xgb

In [4]:
import pandas as pd
import numpy as np

shared_df = pd.read_csv('shared_articles.csv')
shared_df.head()

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
0,1459192779,CONTENT REMOVED,-6451309518266745024,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en


In [5]:
shared_df.dtypes

timestamp           int64
eventType          object
contentId           int64
authorPersonId      int64
authorSessionId     int64
authorUserAgent    object
authorRegion       object
authorCountry      object
contentType        object
url                object
title              object
text               object
lang               object
dtype: object

There are multiple entries for the same content ID which represents the same articles getting shared multiple times. As the article’s content ID is unique, we group them by the content ID

In [6]:
shared_df["text_length"] = shared_df["text"].apply(lambda s : len(s))


df_contentID = shared_df.groupby("contentId")[["timestamp","eventType","contentType","url","title","text","lang","text_length"]].last().reset_index()

df_contentID.head()


,contentId,timestamp,eventType,contentType,url,title,text,lang,text_length
0,-9222795471790223670,1465870439,CONTENT SHARED,HTML,http://idgnow.com.br/mobilidade/2016/06/13/ube...,Uber lança serviço de helicóptero em SP com pr...,"O Uber anunciou nesta segunda-feira, 13/06, o ...",pt,1811
1,-9216926795620865886,1467662560,CONTENT SHARED,HTML,http://maggiesensei.com/2012/11/14/request-les...,Request lesson : How and when to use はず(=hazu)...,= Kotoshi no aki made niwa kare ga dekiru hazu...,la,8915
2,-9194572880052200111,1485266665,CONTENT SHARED,HTML,http://br-arduino.org/2017/01/stemtera-uma-pro...,"STEMTera, uma protoboard com Arduino Uno embutido",Protoboard de 400 pinos + Arduino Uno em um só...,pt,2848
3,-9192549002213406534,1461531427,CONTENT SHARED,HTML,http://thenextweb.com/dd/2016/04/20/googles-ch...,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,en,1117
4,-9190737901804729417,1467653065,CONTENT SHARED,HTML,http://themacro.com/articles/2016/06/the-real-...,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",en,3879


I created a field “num_words_title” which describes the number of words present in the title of the article as there is a tendency of short titled articles getting shared more often or the reverse

In [10]:
# Number of words in the title 
df_contentID["num_words_title"] = df_contentID["title"].apply(lambda s:len([i for i in word_tokenize(s) if i.isalpha()]))

df_contentID.head()

,contentId,timestamp,eventType,contentType,url,title,text,lang,text_length,num_words_title,num_words_text
0,-9222795471790223670,1465870439,CONTENT SHARED,HTML,http://idgnow.com.br/mobilidade/2016/06/13/ube...,Uber lança serviço de helicóptero em SP com pr...,"O Uber anunciou nesta segunda-feira, 13/06, o ...",pt,1811,15,295
1,-9216926795620865886,1467662560,CONTENT SHARED,HTML,http://maggiesensei.com/2012/11/14/request-les...,Request lesson : How and when to use はず(=hazu)...,= Kotoshi no aki made niwa kare ga dekiru hazu...,la,8915,10,1483
2,-9194572880052200111,1485266665,CONTENT SHARED,HTML,http://br-arduino.org/2017/01/stemtera-uma-pro...,"STEMTera, uma protoboard com Arduino Uno embutido",Protoboard de 400 pinos + Arduino Uno em um só...,pt,2848,7,419
3,-9192549002213406534,1461531427,CONTENT SHARED,HTML,http://thenextweb.com/dd/2016/04/20/googles-ch...,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,en,1117,9,190
4,-9190737901804729417,1467653065,CONTENT SHARED,HTML,http://themacro.com/articles/2016/06/the-real-...,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",en,3879,5,637


We need to convert the timestamp from int64 to a date-time type in order to extract the day and hours from it. Feature “day_of_week” and “hour_of_the_day” describe the day of the week on which the article was shared and the hour of the day on which the article was shared. It has been observed that articles shared on weekdays and office hours have high chances of getting viral fast than those shared on weekends or night hours.

In [17]:
# Convert integer timestamp to datetime timestamp
df_contentID['timestamp'] = pd.to_datetime(df_contentID['timestamp'], unit='s')


# Day of the week when the article was shared
df_contentID['day_of_week'] = df_contentID['timestamp'].dt.day_name()

In [21]:

# Time of the day when the article was shared
df_contentID['hour_of_the_day'] = pd.DatetimeIndex(df_contentID['timestamp']).hour

In [22]:
df_contentID.head()

,contentId,timestamp,eventType,contentType,url,title,text,lang,text_length,num_words_title,num_words_text,day_of_week,hour_of_the_day
0,-9222795471790223670,2016-06-14 02:13:59,CONTENT SHARED,HTML,http://idgnow.com.br/mobilidade/2016/06/13/ube...,Uber lança serviço de helicóptero em SP com pr...,"O Uber anunciou nesta segunda-feira, 13/06, o ...",pt,1811,15,295,Tuesday,2
1,-9216926795620865886,2016-07-04 20:02:40,CONTENT SHARED,HTML,http://maggiesensei.com/2012/11/14/request-les...,Request lesson : How and when to use はず(=hazu)...,= Kotoshi no aki made niwa kare ga dekiru hazu...,la,8915,10,1483,Monday,20
2,-9194572880052200111,2017-01-24 14:04:25,CONTENT SHARED,HTML,http://br-arduino.org/2017/01/stemtera-uma-pro...,"STEMTera, uma protoboard com Arduino Uno embutido",Protoboard de 400 pinos + Arduino Uno em um só...,pt,2848,7,419,Tuesday,14
3,-9192549002213406534,2016-04-24 20:57:07,CONTENT SHARED,HTML,http://thenextweb.com/dd/2016/04/20/googles-ch...,Chrome OS now has Material Design for the desktop,Chromebook users are in for a surprise: Chrome...,en,1117,9,190,Sunday,20
4,-9190737901804729417,2016-07-04 17:24:25,CONTENT SHARED,HTML,http://themacro.com/articles/2016/06/the-real-...,The Real Product Market Fit,"Michael Seibel, Y Combinator Partner, on knowi...",en,3879,5,637,Monday,17


I have created a list with the names of the popular blogs where the value >= 1000 and then we divide the articles based on whether it was shared on a popular blog site or not

In [23]:
def pop_blog(blog):
    if blog in pop_blogs:
        return blog
    else:
        return "not_pop"


# Whether the content is posted in popular blog pages like medium
pop_blogs = list(df_contentID["url"].apply(lambda s : s.split("//")[1].split("/")[0]).value_counts().apply(lambda s : s >= 1000)[0:6].index)
df_contentID["blog"] = df_contentID["url"].apply(lambda s : s.split("//")[1].split("/")[0])
df_contentID["in_pop_blog"] = df_contentID["blog"].apply(pop_blog)

I also divided the article based on the sentiment into positive or negative. I have used the TextBlob package for this implementation

In [ ]:
def get_content_sentiment(tweet):
    try:
        analysis = TextBlob(tweet) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
    except Exception as e:
        print(e)
        return "NA"


# Sentiment of the content
df_contentID["content_sentiment"] = df_contentID["text"].apply(lambda s:get_content_sentiment(s))

In [26]:
users_df = pd.read_csv('users_interactions.csv')
users_df.head()

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN


In [27]:
users_df.columns = ["timestamp","eventType","contentId","personId","sessionId","userAgent","userRegion","userCountry"]
users_df['timestamp'] = pd.to_datetime(users_df['timestamp'], unit='s')

In [29]:
mergerd_data = pd.merge(users_df,df_contentID,how="left",on="contentId")
mergerd_data.rename(columns={'timestamp_x': 'shared_date','timestamp_y': 'read_date'}, inplace=True)

mergerd_data.dtypes


shared_date          datetime64[ns]
eventType_x                  object
contentId                     int64
personId                      int64
sessionId                     int64
userAgent                    object
userRegion                   object
userCountry                  object
read_date            datetime64[ns]
eventType_y                  object
contentType                  object
url                          object
title                        object
text                         object
lang                         object
text_length                   int64
num_words_title               int64
num_words_text                int64
day_of_week                  object
hour_of_the_day               int64
blog                         object
in_pop_blog                  object
content_sentiment            object
dtype: object

In [30]:
mergerd_data["read_diff_min"] = mergerd_data["read_date"] - mergerd_data["shared_date"]
mergerd_data["read_diff_min"] = mergerd_data["read_diff_min"]/np.timedelta64(1,'m')
mergerd_data.head()

,shared_date,eventType_x,contentId,personId,sessionId,userAgent,userRegion,userCountry,read_date,eventType_y,contentType,url,title,text,lang,text_length,num_words_title,num_words_text,day_of_week,hour_of_the_day,blog,in_pop_blog,content_sentiment,read_diff_min
0,2016-06-08 19:10:32,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,2016-06-07 14:23:54,CONTENT SHARED,HTML,http://techcrunch.com/2016/06/07/hiri/,Hiri wants to fix the workplace email problem,Hiri is the latest startup trying to fix email...,en,3881,8,653,Tuesday,14,techcrunch.com,techcrunch.com,positive,-1726.633333
1,2016-06-08 19:02:40,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,2016-06-06 14:29:28,CONTENT SHARED,HTML,https://www.nngroup.com/articles/top-intranet-...,Top 10 Intranet Trends of 2016,"Summary: Hero images, carousels, fat footers, ...",en,9244,4,1407,Monday,14,www.nngroup.com,not_pop,positive,-3153.200000
2,2016-06-08 20:03:10,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,2016-06-08 14:34:51,CONTENT SHARED,HTML,http://blog.runrun.it/erros-de-portugues-em-e-...,71 erros de português que precisam sumir dos s...,Escrever um e-mail não deveria ser uma coisa t...,pt,12440,8,2048,Wednesday,14,blog.runrun.it,not_pop,negative,-328.316667
3,2016-06-08 19:24:55,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,2016-06-08 14:34:51,CONTENT SHARED,HTML,http://blog.runrun.it/erros-de-portugues-em-e-...,71 erros de português que precisam sumir dos s...,Escrever um e-mail não deveria ser uma coisa t...,pt,12440,8,2048,Wednesday,14,blog.runrun.it,not_pop,negative,-290.066667
4,2016-06-08 18:58:10,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,2016-06-08 18:48:44,CONTENT SHARED,HTML,http://www.fastcompany.com/3058876/your-most-p...,How This Googler Redesigned The Workweek,Jake Knapp has always been concerned about the...,en,6713,6,1167,Wednesday,18,www.fastcompany.com,not_pop,positive,-9.433333


After merging the data we separate the two timestamps a shared date and read_date. I calculated the difference between the two in order to check if the article has been shared within 2 hours of it being read. This is an important feature as fast sharing leads to article getting viral quickly

In [31]:
# Number of views of the article within 120 minutes of sharing
num_page_views = mergerd_data.groupby("contentId")["read_diff_min"].apply(list).apply(lambda s:len([i for i in s if i <=120]))

I have created a new data frame called per_article which is used to calculate the virality score of each article based on the given formula 

In [34]:
# Features per article
per_article = pd.crosstab(mergerd_data['contentId'], mergerd_data['eventType_x']).reset_index()
per_article = pd.merge(per_article,num_page_views,on="contentId",how="inner")
per_article = pd.merge(per_article, df_contentID[["contentId","content_sentiment","lang","text_length","num_words_title","day_of_week","hour_of_the_day","in_pop_blog"]], on="contentId",how="inner")

# target variable
per_article["virality"] = 1 * per_article["VIEW"] + 4 * per_article["LIKE"] + 10 * per_article["COMMENT CREATED"] + 25 * per_article["FOLLOW"] + 100 * per_article["BOOKMARK"]
per_article.rename(columns={'read_diff_min': 'num_page_views'}, inplace=True)

In [35]:
per_article.head()

,contentId,BOOKMARK,COMMENT CREATED,FOLLOW,LIKE,VIEW,num_page_views,content_sentiment,lang,text_length,num_words_title,day_of_week,hour_of_the_day,in_pop_blog,virality
0,-9222795471790223670,1,2,3,4,16,26,neutral,pt,1811,15,Tuesday,2,not_pop,227
1,-9216926795620865886,1,1,1,3,15,21,positive,la,8915,10,Monday,20,not_pop,162
2,-9194572880052200111,2,1,1,4,21,29,negative,pt,2848,7,Tuesday,14,not_pop,272
3,-9192549002213406534,0,1,0,5,50,56,positive,en,1117,9,Sunday,20,not_pop,80
4,-9190737901804729417,0,0,0,1,8,9,positive,en,3879,5,Monday,17,not_pop,12


v

In [37]:
# Feature transformation
per_article["text_length"] = np.log(per_article["text_length"])

# One hot encoding of categorical
per_article = pd.get_dummies(per_article)

In [38]:
# Drop columns which are collinear
features = per_article.drop(["contentId","lang_pt","virality","BOOKMARK","COMMENT CREATED","FOLLOW","LIKE","VIEW","in_pop_blog_not_pop"],axis=1)
target = per_article["virality"]

print(features.shape, target.shape)
features.head()

(2987, 24) (2987,)


,num_page_views,text_length,num_words_title,hour_of_the_day,content_sentiment_negative,content_sentiment_neutral,content_sentiment_positive,lang_en,lang_es,lang_ja,lang_la,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday,in_pop_blog_cloudplatform.googleblog.com,in_pop_blog_exame.abril.com.br,in_pop_blog_medium.com,in_pop_blog_startupi.com.br,in_pop_blog_techcrunch.com,in_pop_blog_www.imdb.com
0,26,2.015121,15,2,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,21,2.207779,10,20,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,29,2.073722,7,14,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,56,1.948536,9,20,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,9,2.111828,5,17,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [42]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)
print(list(features.columns))
print("\n")
regr = LinearRegression()
regr.fit(X_train, y_train)
pred = regr.predict(X_test)
mse = metrics.mean_squared_error(y_test,pred)
print('Intercept: ', regr.intercept_)
print('R^2 score: ',regr.score(X_train, y_train))
print('MSE: ',mse)
print("RMSE: %.2f" % (mse*(1/2.0)))
print('test score: ',regr.score(X_test, y_test))

['num_page_views', 'text_length', 'num_words_title', 'hour_of_the_day', 'content_sentiment_negative', 'content_sentiment_neutral', 'content_sentiment_positive', 'lang_en', 'lang_es', 'lang_ja', 'lang_la', 'day_of_week_Friday', 'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday', 'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday', 'in_pop_blog_cloudplatform.googleblog.com', 'in_pop_blog_exame.abril.com.br', 'in_pop_blog_medium.com', 'in_pop_blog_startupi.com.br', 'in_pop_blog_techcrunch.com', 'in_pop_blog_www.imdb.com']


Intercept:  -396.7998415935646
R^2 score:  0.47838803486688575
MSE:  21562.61399117865
RMSE: 10781.31
test score:  0.5711546731313162


In [46]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

regressor = SVR(kernel='rbf')
regressor.fit(X_train,y_train)

y_pred = regressor.predict(X_test)
print('Intercept: ', regressor.intercept_)
print('R^2 score: ',regressor.score(X_train, y_train))
print('MSE: ',metrics.mean_squared_error(y_test,pred))
print("RMSE: %.2f" % (mse*(1/2.0)))
print('test score: ',regressor.score(X_test, y_test))

Intercept:  [146.04703828]
R^2 score:  0.14367483691597704
MSE:  21696.726217921077
RMSE: 10848.36
test score:  0.19037280918120592


In [53]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators=100, random_state=0)
#regressor.fit(train_x, train_y) #Training the model over the training data with the below mentioned parameters

model = regressor.fit(X_train, y_train)

print(model.score(X_train, y_train))

print(model.score(X_test, y_test))

ypred = model.predict(X_test)
print("RMSE: %.2f", metrics.mean_squared_error(y_test, ypred))

0.9299330915510764
0.4110378184879043
RMSE: %.2f 29613.390608862876


In [50]:
from sklearn.model_selection import GridSearchCV

xgbr = xgb.XGBRegressor(verbosity=0)
# print(xgbr)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07, 0.1, 0.3], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7, 0.9],
              'colsample_bytree': [0.8, 0.9],
              'n_estimators': [500,800,1000,1200]}

xgb_grid = GridSearchCV(xgbr,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(X_train,y_train)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 2 folds for each of 240 candidates, totalling 480 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  40 tasks      | elapsed:  1.6min


0.3758550324450568
{'colsample_bytree': 0.9, 'learning_rate': 0.03, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.9}


In [51]:
score = xgb_grid.score(X_train, y_train)
y_pred = xgb_grid.predict(X_test)
mse = metrics.mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
print('R^2 score: ',xgb_grid.score(X_train, y_train))
print("RMSE: %.2f" % (mse*(1/2.0)))
print('test score: ',xgb_grid.score(X_test, y_test))

MSE: 22190.84
R^2 score:  0.9191180124584104
RMSE: 11095.42
test score:  0.5586603683919096
